In [1]:
# General Imports
import os
import pandas as pd
import numpy as np
from rdkit import Chem
import time
import sys
from pathlib import Path
SELFIES_coder_path = Path("../SELFIES_coder")
sys.path.append(SELFIES_coder_path.as_posix())
import SELFIES_coder as SELFIES_CODER
import selfies as sf

SELFIES_CODER.TEST()

# Inital time
start = time.time()


# Know where user actually is
directory_path = os.getcwd()
print("My current directory is : " + directory_path)
folder_name = os.path.basename(directory_path)
print("My directory name is : " + folder_name)

#load file with smiles codes to train and test (in this case one column is enough)
smifile = ('../Data/training_data_smiles_caffeine_reduced.parquet')

data = pd.read_parquet(smifile)


from sklearn.model_selection import train_test_split
smiles_train, smiles_test = train_test_split(data["SMILES"], test_size=0.1, train_size=0.9, random_state=42)
print(smiles_train.shape)
print(smiles_test.shape)


print(data[:10])


data = SELFIES_CODER.get_encoded_SELFIES(data['SMILES'].to_list())

print(data[2])


# characters that are used in given SMILES dataset along with initial and stopping characters
charset = set("".join(list(data[0]))+"!E")
char_to_int = dict((c,i) for i,c in enumerate(charset))
int_to_char = dict((i,c) for i,c in enumerate(charset))
embed = data[3] + 5 #20
print(str(charset))
print(len(charset), embed)

shap = smiles_train.shape[0]
shap1 = smiles_test.shape[0]


import json
json = json.dumps(data[2])
f = open("SELFIES_to_mol_seq.json","w")
f.write(json)
f.close()

import json 
json = json.dumps(data[1])
f = open("mol_seq_to_SELFIES.json","w")
f.write(json)
f.close()

import json

json = json.dumps(char_to_int)

# open file for writing, "w" 
f = open("mol_seq_to_int.json","w")
f.write(json)
f.close()

import json
json1 = json.dumps(int_to_char)

# open file for writing, "w" 
f = open("int_to_mol_seq.json","w")
f.write(json1)
f.close()


#vectorization of molecular sequence code
def vectorize(smiles, shap):
        one_hot =  np.zeros((shap, embed , len(charset)),dtype=np.int8)
        for i,smile in enumerate(smiles):
            #encode the startchar
            one_hot[i,0,char_to_int["!"]] = 1
            #encode the rest of the chars
            for j,c in enumerate(smile):
                one_hot[i,j+1,char_to_int[c]] = 1
            #Encode endchar
            one_hot[i,len(smile)+1:,char_to_int["E"]] = 1
        #Return two, one for input and the other for output
        return one_hot[:,0:-1,:], one_hot[:,1:,:]
smiles_train_ = data[0][:int(shap)]
smiles_test_ = data[0][int(shap):int(shap+shap1)]
X_train, Y_train = vectorize(smiles_train_, shap)
X_test,Y_test = vectorize(smiles_test_, shap1)
print(smiles_train.iloc[0])


#going back from vectorized form to redable string
string_test = "".join([int_to_char[idx] for idx in np.argmax(X_train[0,:,:], axis=1)])
print(string_test)

decoded = SELFIES_CODER.convert_back_to_SEFLIES(string_test, data[1])
print(decoded)

decoded_two = SELFIES_CODER.convert_back_to_SEFLIES(smiles_train_[0], data[1])
print(decoded_two)

print("Correct encoding-decoding: "+str(decoded == decoded_two))

smiles_train_[0]
#Import Keras objects
#LSTM cells special kinds of neural network units that are designed to keep an internal state for longer iterations
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Concatenate
from keras import regularizers
input_shape = X_train.shape[1:]
output_dim = Y_train.shape[-1]
latent_dim = 128
lstm_dim = 128



#encoder-decoder architecture
unroll = False
encoder_inputs = Input(shape=input_shape)
encoder = LSTM(lstm_dim, return_state=True,
                unroll=unroll)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
states = Concatenate(axis=-1)([state_h, state_c])
neck = Dense(latent_dim, activation="relu")
neck_outputs = neck(states)



decode_h = Dense(lstm_dim, activation="relu")
decode_c = Dense(lstm_dim, activation="relu")
state_h_decoded =  decode_h(neck_outputs)
state_c_decoded =  decode_c(neck_outputs)
encoder_states = [state_h_decoded, state_c_decoded]
decoder_inputs = Input(shape=input_shape)
decoder_lstm = LSTM(lstm_dim,
                    return_sequences=True,
                    unroll=unroll
                   )
decoder_outputs = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(output_dim, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
#Define the model, that inputs the training vector for two places, and predicts one character ahead of the input
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
print(model.summary())


from keras.callbacks import History, ReduceLROnPlateau
h = History()
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=15, min_lr=0.000001, verbose=1, min_delta=1e-5) #patience=10? #epsilon=min_delta



from tensorflow.keras.optimizers import RMSprop, Adam
opt=Adam(learning_rate=0.005) #Default 0.001
model.compile(optimizer=opt, loss='categorical_crossentropy')



start_MODEL = time.time()
model.fit([X_train,X_train],Y_train, epochs=200, batch_size=256, shuffle=True, callbacks=[h, rlr], validation_data=([X_test,X_test],Y_test)) #100
end_MODEL = time.time()
print("The time of execution of above program is :", end_MODEL-start_MODEL)


import pickle
f = open("Neural_network_history.pickle","wb")
pickle.dump(h.history, f)


smiles_to_latent_model = Model(encoder_inputs, neck_outputs)

smiles_to_latent_model.save("mol_seq2lat.h5")


latent_input = Input(shape=(latent_dim,))
#reuse_layers
state_h_decoded_2 =  decode_h(latent_input)
state_c_decoded_2 =  decode_c(latent_input)
latent_to_states_model = Model(latent_input, [state_h_decoded_2, state_c_decoded_2])
latent_to_states_model.save("lat2state.h5")


#Last one is special, we need to change it to stateful, and change the input shape
inf_decoder_inputs = Input(batch_shape=(1, 1, input_shape[1]))
inf_decoder_lstm = LSTM(lstm_dim,
                    return_sequences=True,
                    unroll=unroll,
                    stateful=True
                   )
inf_decoder_outputs = inf_decoder_lstm(inf_decoder_inputs)
inf_decoder_dense = Dense(output_dim, activation='softmax')
inf_decoder_outputs = inf_decoder_dense(inf_decoder_outputs)
sample_model = Model(inf_decoder_inputs, inf_decoder_outputs)


#Transfer Weights
for i in range(1,3):
    sample_model.layers[i].set_weights(model.layers[i+6].get_weights())
sample_model.save("samplemodel.h5")



print(sample_model.summary())


end = time.time()


print("The time of execution of above program is :", end-start)

working
My current directory is : C:\Users\aleks\Documents\GitHub\caffeine_new_structures\Neural_network
My directory name is : Neural_network
(118386,)
(13154,)
                                                   SMILES
99343   N(CCN1C(CCC1=O)=O)C2=NC=3N(C)C(=O)N(C(=O)C=3N2C)C
136644  S/C(C(OC)=O)=C\C(=CN(C)CCCCCCN(C)C1=NC2=C(N1C)...
236654  N1=C(N(C=2C(N(C(N(C)C1=2)=O)C)=O)C)NCC/N=C3/SC...
131075  C(CCCNC=1N(C)C=2C(N(C(=O)N(C)C=2N=1)C)=O)NC(CS...
250375        C(NC1=NC=2N(C)C(N(C(C=2N1C)=O)C)=O)CNC(=O)C
163202  CN1C(N(C(=O)C=2N(C(NCCN3C(=O)C=CC3=O)=NC1=2)C)...
307481  C(SCC(=O)NCCCNC=1N(C)C=2C(N(C)C(=O)N(C)C=2N=1)...
125684  N(C=1N(C2=C(N(C)C(=O)N(C2=O)C)N=1)C)CC/N=C3\NC...
138992  C(N(C)C1=NC=2N(C(N(C(=O)C=2N1C)C)=O)C)CCCCCN(C...
39173   C(CN1C(=O)C2=CC=CC=C2C1=O)NC=3N(C)C4=C(N=3)N(C...
All works correct, encoding leads to the same data during decoding...
{'[=N+1]': '#', '[C]': '$', '[N]': '%', '[N-1]': '&', '[=Ring1]': "'", '[\\S]': '(', '[\\N]': ')', '[O-1]': '*', '[Ring2]': '+', '

Epoch 35/200
463/463 [==============================] - 11s 23ms/step - loss: 0.1077 - val_loss: 0.1073
Epoch 36/200
463/463 [==============================] - 11s 23ms/step - loss: 0.1061 - val_loss: 0.1223
Epoch 37/200
463/463 [==============================] - 11s 23ms/step - loss: 0.1065 - val_loss: 0.1077
Epoch 38/200
463/463 [==============================] - 11s 23ms/step - loss: 0.1015 - val_loss: 0.1037
Epoch 39/200
463/463 [==============================] - 11s 23ms/step - loss: 0.1017 - val_loss: 0.1006
Epoch 40/200
463/463 [==============================] - 11s 23ms/step - loss: 0.0987 - val_loss: 0.1033
Epoch 41/200
463/463 [==============================] - 11s 23ms/step - loss: 0.1027 - val_loss: 0.0984
Epoch 42/200
463/463 [==============================] - 11s 23ms/step - loss: 0.0968 - val_loss: 0.1050
Epoch 43/200
463/463 [==============================] - 11s 23ms/step - loss: 0.0944 - val_loss: 0.1167
Epoch 44/200
463/463 [==============================] - 11s 23ms

463/463 [==============================] - 10s 22ms/step - loss: 0.0497 - val_loss: 0.0570
Epoch 114/200
463/463 [==============================] - 10s 22ms/step - loss: 0.0474 - val_loss: 0.0537
Epoch 115/200
463/463 [==============================] - 10s 22ms/step - loss: 0.0477 - val_loss: 0.0560
Epoch 116/200
463/463 [==============================] - 10s 22ms/step - loss: 0.0473 - val_loss: 0.0497
Epoch 117/200
463/463 [==============================] - 10s 22ms/step - loss: 0.0504 - val_loss: 0.0531
Epoch 118/200
463/463 [==============================] - 10s 22ms/step - loss: 0.0474 - val_loss: 0.0506
Epoch 119/200
463/463 [==============================] - 10s 22ms/step - loss: 0.0461 - val_loss: 0.0524
Epoch 120/200
463/463 [==============================] - 10s 22ms/step - loss: 0.0462 - val_loss: 0.0478
Epoch 121/200
463/463 [==============================] - 10s 22ms/step - loss: 0.0456 - val_loss: 0.0476
Epoch 122/200
463/463 [==============================] - 10s 22ms/ste

463/463 [==============================] - 11s 23ms/step - loss: 0.0331 - val_loss: 0.0354
Epoch 192/200
463/463 [==============================] - 11s 23ms/step - loss: 0.0310 - val_loss: 0.0364
Epoch 193/200
463/463 [==============================] - 11s 23ms/step - loss: 0.0325 - val_loss: 0.0340
Epoch 194/200
463/463 [==============================] - 11s 23ms/step - loss: 0.0294 - val_loss: 0.0360
Epoch 195/200
463/463 [==============================] - 11s 24ms/step - loss: 0.0315 - val_loss: 0.0367
Epoch 196/200
463/463 [==============================] - 11s 23ms/step - loss: 0.0319 - val_loss: 0.0361
Epoch 197/200
463/463 [==============================] - 11s 23ms/step - loss: 0.0310 - val_loss: 0.0372
Epoch 198/200
463/463 [==============================] - 11s 23ms/step - loss: 0.0305 - val_loss: 0.0325
Epoch 199/200
463/463 [==============================] - 11s 23ms/step - loss: 0.0304 - val_loss: 0.0427
Epoch 200/200
463/463 [==============================] - 11s 23ms/ste

In [2]:
import pickle #if the history file is empty
f = open("Neural_network_history.pickle","wb")
pickle.dump(h.history, f)